# Défi Grosses Data 2018

## Début Commun pour tous

In [8]:
import numpy as np
import pandas as pd
import datetime as dt
from os import listdir
import Annex

Importe les données concaténées:

In [9]:
df=Annex.get_data_raw()
df.shape

(189280, 31)

Importe les données séparées par type:

In [10]:
meteo_quant, meteo_qual, meteo_date, meteo_y=Annex.get_data_tidied()

Remplace les variables qualitatives par leur indicatrices. 

**Attention, ici seuls "mois" et "insee" sont considérés qualitatives!** (what about the wind? what about the "ech"?)

In [11]:
df_dummies=pd.get_dummies(df[['mois','insee']])
df_full_qtt=pd.concat([df,df_dummies],axis=1)
df_full_qtt=df_full_qtt.drop(['mois','insee'],axis=1)

Sépare les échantillons d'apprentissage et de test

In [12]:
from sklearn.model_selection import train_test_split
Y=df_full_qtt['tH2_obs']
X=df_full_qtt
X=X.drop(['tH2_obs','date'],axis=1) ## !!! Date?
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.3,random_state=11)
X_train.shape, X_test.shape

((132496, 46), (56784, 46))

## Maintenant, faites votre vie!